In [135]:
import csv
import random
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt

class LVQ(): 
    def __init__(self, nome_arquivo):
        """
        Construtor da classe
        :param nome_arquivo: nome do arquivo csv que contem os dados
        """
        self.dados = []
        self.qtd_caracteristicas = 0
        self.amplitudes = []
        self.carregar("IRIS.csv", "numeros")
        self.qtd_caracteristicas = len(self.dados[0])-1
        
    def carregar(self, arquivo_csv: str="IRIS.csv", tipo_caracteristicas: str="numeros"):
        """
        Carrega os dados iniciais da classe através de um arquivo csv
        :param arquivo_csv: nome do arquivo csv
        :param tipo_caracteristicas: Tipo das caracteristicas do conjunto de dados
        """
        with open(arquivo_csv, 'r') as arquivo_csv:
            arquivo = csv.reader(arquivo_csv)
            if tipo_caracteristicas == "numeros":
                for index, linha in enumerate(arquivo):
                    if linha:
                        self.dados += [list(map(float, linha[0:-1]))]
                        self.dados[index] += [linha[-1]]
            elif tipo_caracteristicas == "string":
                for index, linha in enumerate(arquivo):
                    if linha:
                        dados = 1
                        self.dados += linha
        
    def normalizar(self):
        """
        Normalizada todas as caracteristicas para um intervalo de 0 - 1, para todas tenham o mesmo peso na 
        classificacao
        """
        lista = []*(len(self.dados[0])-1)
        self.amplitudes = []
        for caracteristica in range(len(self.dados[0])-1):
            lista = [elemento[caracteristica] for elemento in self.dados]
            self.amplitudes += [[max(lista), min(lista)]]
            for elemento in self.dados:
                elemento[caracteristica] = (elemento[caracteristica] - min(lista))/(max(lista)+min(lista))
        
    def triagem(self, split: float=0.65):
        """
        Divide aleatoriament os elementos do conjunto de dados em dois subconjuntos: teste e treino
        :param split: de 0 a 1 -> 'porcentagem' dos elementos que serao do conjunto de treino
        """        
        self.treino, self.teste = [], []
        for elemento in self.dados:
            if random.random() < split:
                self.treino += [elemento]
            else:
                self.teste += [elemento]
    
    def resumir(self, n: float=10,  e: float=10, t: float=0.4):
        """
        Retorna o codebook dos dados, ou seja, os elementos que melhor representam o todo
        :param t: taxa de aprendizado inicial
        :param e: numero de epocas
        :param n: numero de elementos do coodbook 
        """
        #Geracacao aleatorio dos elementos iniciais do codebook         
        self.codebook = [[]]*n
        for i in range(n):
            self.codebook[i] = [0] * (self.qtd_caracteristicas + 1)
            for caracteristica in range(self.qtd_caracteristicas + 1):
                self.codebook[i][caracteristica] = random.choice(self.dados)[caracteristica]

        for epoca in range(e):
            taxa = t * (1.0-(epoca/float(e)))
            for elemento in self.treino:
                representante = self.encontrar_mais_proximo(elemento, self.codebook)
                o = -1
                if representante[-1] == elemento[-1]:
                    o = 1
                for caracteristica in range(self.qtd_caracteristicas):
                    erro = (elemento[caracteristica]-representante[caracteristica]) 
                    representante[caracteristica] += (erro * taxa * o)
      
    def testar(self):
        """
        Executa a classificacao para cada elemento do conjunto teste e retorna a precisao do algoritmo
        """
        qtd_teste = len(self.teste)
        precisao = 100.0
        for elemento in self.teste: 
            bmu = self.encontrar_mais_proximo(elemento, self.codebook)
            if bmu[-1] != elemento[-1]:
                precisao -= (1/qtd_teste)*100
 
        return precisao
    
    def encontrar_mais_proximo(self, elemento, lista):
        """
        Executa a classificacao para cada elemento do conjunto teste e retorna a precisao do algoritmo
        :param elemento: vetor para o qual deve-se vetor mais proximo de uma dada lista
        :param lista: lista de vetores
        """
        resposta = [lista[0], spatial.distance.euclidean(elemento[0:-1], lista[0][0:-1])]
        for i in lista:
            distancia = spatial.distance.euclidean(elemento[0:-1], i[0:-1])
            if distancia < resposta[1]: 
                resposta = [i, distancia]
        return resposta[0]
    
    @property
    def representantes(self):
        """
        Retorna o codebook "original", com as caracteristicas em seus intervalos originais. Ou seja, 
        retorna o codebook desnormalizado, caso ele tenha sido normalizado
        """
        representantes_desnormalizados = [[]]*len(self.codebook)
        if self.amplitudes:
            for index, representante in enumerate(self.codebook): 
                representante_desnormalizado = []
                for caracteristica in range(self.qtd_caracteristicas):
                    aux = ((self.amplitudes[caracteristica][0] + self.amplitudes[caracteristica][1])\
                          * representante[caracteristica]) + self.amplitudes[caracteristica][1]
                    representante_desnormalizado += [aux]
                representante_desnormalizado += [representante[-1]]    
                representantes_desnormalizados[index] = representante_desnormalizado
        else: 
            return self.codebook
        return representantes_desnormalizados
    

# Dados normalizados
print("Algoritmo com os dados normalizados entre 0 - 1")
iris_norm = LVQ("IRIS.csv")
iris_norm.triagem(0.75)
iris_norm.normalizar()
iris_norm.resumir(n=10, e=13, t=0.4)
# for representante in iris_norm.representantes:
#     print(representante)
print("Precisão: ", iris_norm.testar(), "% \n")

# Sem normalização 
print("Algoritmo com os dados não normalizados")
iris = LVQ("IRIS.csv")
iris.triagem(0.75)
iris.resumir(n=10, e=13, t=0.4)
print("Precisão: ", iris.testar(),"% \n")
# for representante in iris.representantes:
#     print(representante)
    

Algoritmo com os dados normalizados entre 0 - 1
Precisão:  100.0 % 

Algoritmo com os dados não normalizados
Precisão:  95.23809523809524 % 

